In [3]:
import time
import random
import pygame
import numpy as np
import subprocess
from screeninfo import get_monitors
import math
import csv
from datetime import datetime
import os
import glob

def calc_speed(spd, dfs, fps, disp):
    #To calculate visual angle for any given display setup
    
    speed = np.array(spd) * np.pi / 180 #numpy arrray of speed in visual angle
    tsp = np.tan(speed)
    ss = np.array(mon_size()[disp][0]) #screen size in cms
    sr = np.array(mon_size()[disp][1])
    sw_r = round(sr[0]/ss[0], 2)
    sh_r = round(sr[1]/ss[1], 2)
        
    sx = int(tsp[0] * dfs * sw_r / fps)
    sy = int(tsp[1] * dfs * sh_r / fps)
    asx = np.arctan(sx * fps / (sw_r * dfs)) * 180 / np.pi
    asy = np.arctan(sy * fps / (sh_r * dfs)) * 180 / np.pi
    
    s_ball = [sx, sy]
    av_ball = [asx, asy]
    
    return s_ball, av_ball

def mon_size():
    #Gets details of all displays connected to system
    
    dml = []
    for m in get_monitors():
        wl = m.width_mm; hl = m.height_mm;
        wp = m.width; hp = m.height;
        dml.append([[round(wl,1), round(hl,1)], [wp, hp]])
    return dml

def calibration_points(rb, disp):
    # Generates fifteen points that are used as start/fixation points for the given display
    
    disp_size = mon_size()[disp]
    ssize = np.array(mon_size()[disp][0]) 
    srez = np.array(mon_size()[disp][1])
    diam = rb*2

    binelh, binerh = [diam, srez[0]-diam]
    bineuv, binedv = [diam, srez[1]-diam]

    midh = math.ceil(srez[0]/2)
    midv = math.ceil(srez[1]/2)

    midlh = math.ceil((midh-diam)/2 + diam)
    midrh = math.ceil((binerh - midh)/2 + midh)
    # midlv = math.ceil((midh-rb)/2 + rb)
    # midrv = math.ceil((bin_edge_h - midh)/2)

    calib_pts = []

    calib_pts.append([binelh, bineuv])
    calib_pts.append([midlh, bineuv])
    calib_pts.append([midh, bineuv])
    calib_pts.append([midrh, bineuv])
    calib_pts.append([binerh, bineuv])

    calib_pts.append([binelh, midv])
    calib_pts.append([midlh, midv])
    calib_pts.append([midh, midv])
    calib_pts.append([midrh, midv])
    calib_pts.append([binerh, midv])

    calib_pts.append([binelh, binedv])
    calib_pts.append([midlh, binedv])
    calib_pts.append([midh, binedv])
    calib_pts.append([midrh, binedv])
    calib_pts.append([binerh, binedv])
    
    dens = ssize/srez
    bin_size = [(binerh-binelh)/4, (binedv - bineuv)/2] #Calculates the distance between each fixation point 
    
    return calib_pts, bin_size

def exit_disp(all_points, correct_clicks, wrong_clicks, tooerc):
    #displays results for a few seconds
    
    screen.fill((0,0,0))
    font = pygame.font.SysFont(None, 48)
    text = font.render(f"Correct guess = {correct_clicks}", True, (255, 255, 255))
    text2 = font.render(f"Wrong guess = {wrong_clicks}", True, (255, 255, 255))
    text3 = font.render(f" Too hasty = {tooerc}", True, (255, 255, 255))
    screen.blit(text, (width/2 - text.get_width()/2, height/2 - 3 * text.get_height()/2))
    screen.blit(text2, (width/2 - text2.get_width()/2, height/2))
    screen.blit(text3, (width/2 - text3.get_width()/2, height/2 + 3 * text3.get_height()/2))
    pygame.display.flip()

    pygame.time.wait(1000)
    
    traj = np.array(all_points)
    now = datetime.now();today = now.strftime("%d-%m-%Y-%H-%M-%S") #Collects current date and time
    
    if not os.path.exists('Trajectories'):
        os.makedirs('Trajectories')
    
    fl_name = 'Trajectories/Trajectory_' + today + '.csv'
    
    # open the file in the write mode
    f = open(fl_name, 'w')

    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    for i in traj:
        writer.writerow(i)

    # close the file
    f.close()
    
    print(traj)
    
    pygame.display.quit()
    pygame.quit()
    exit()

    
# initialize pygame, sound player
pygame.init()
pygame.mixer.init()

deets = pygame.display.Info() #Collects the resolution of your screen 


#----------------EDIT HERE ONLY---------------------------------------------------------------------------------------------------------------------------------

vol = 0.1          #set volume
ntrial = 1         #number of trials
rb = 40            #radius of the ball
app_dur = [3, 7]   #time for which ball is seen
fl_dur = 0.15      #Flicker duration
click_window = 1   #Time after flicker within which the subject must respond 
set_fps = 60       #Set fps of game
rend_stats = True  #render fps and time elapsed 
vas = [10, 10]     #set speed in visual angle/sec 
dfs = 800          #Determine approximte distance from monitor
disp = 0           #Set output display
audio = True       #Whether you want to play audio or not


# define colors 
col_bb = (255, 0, 0)                   #colour of ball when bouncing
col_flk = (150, 0, 50)                 #colour of ball during flicker
col_bg = (0, 0, 0)                     #colour of background
fixation_color = (255, 255, 255)       #colour of ball at the start of trial - fixation point

#----------------------------------------------------------------------------------------------------------------------------------------------------------------


trial_order_l, size_bin = calibration_points(rb, disp)
trial_order = np.array(trial_order_l)
np.random.shuffle(trial_order)

# Calculates the pixel speed based on your choice of visual angle speed. 
# NOTE - you need to specify an approximate distance from screen. 
spd, avas = calc_speed(vas, dfs, set_fps, disp) 

print(size_bin)

# defining width and height of screen - Collected from the info taken of the screen.
infoObject = pygame.display.Info()
width = infoObject.current_w
height = infoObject.current_h
screen_res = (width, height)

if audio:
    right_ans = pygame.mixer.Sound("/home/yramakrishna/DeepLabCut/conda-environments/Sounds/reward.wav")
    wrong_ans = pygame.mixer.Sound("/home/yramakrishna/DeepLabCut/conda-environments/Sounds/failure.wav")
    right_ans.set_volume(vol)
    wrong_ans.set_volume(vol)


clock = pygame.time.Clock()

pygame.display.set_caption("GFG Bouncing game")
screen = pygame.display.set_mode((screen_res), pygame.FULLSCREEN, display=disp) #renders it to the entire screen


correct_clicks = 0
wrong_clicks = 0
tooerc = 0
trial = 1

llb, lub = app_dur

timec = pygame.time.get_ticks()
quits = False

last_seen = []
all_points = []

while len(trial_order)>0:
    # Trial runs
    
    speed = list(map(abs, spd))
    
    rand_dir = random.randint(1, 4)
    if rand_dir == 1:
        speed = [spd[0], spd[1]]
    if rand_dir == 2:
        speed = [-spd[0], spd[1]]
    if rand_dir == 3:
        speed = [spd[0], -spd[1]]
    if rand_dir == 4:
        speed = [-spd[0], -spd[1]]
    
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                exit_disp(all_points, correct_clicks, wrong_clicks, tooerc)
    
    pygame.event.clear()
    rspt = random.randint(0, len(trial_order)-1)
    # Define the fixation point coordinates
    fixation = trial_order[rspt]
    print(fixation)
    fixation_x, fixation_y = fixation #randomize this
    fixation_radius = rb
    
    collect_pts = []

    # Draw the fixation point
    pygame.draw.circle(screen, fixation_color, (fixation_x, fixation_y), fixation_radius)

    # Update the display
    pygame.display.flip()

    # Wait for a mouse click on the fixation point to start the trial
    trial_started = False
    while not trial_started:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    trial_started = True
                if event.key == pygame.K_ESCAPE:
                    exit_disp(all_points, correct_clicks, wrong_clicks, tooerc)
                    
    # define ball
    ball_obj = pygame.draw.circle(surface=screen, color=col_bb, center=fixation, radius=rb)
    
    start_pt = fixation
    
    # speed = [X direction speed, Y direction speed] in pixels per frame
    speed = spd

    limit = random.randint(llb, lub) 
    
    start_time = time.time()
    flick_em1 = random.randint(0,1)
    flick_em = bool(flick_em1)
    flickr = limit-(fl_dur+click_window)
    bound = [start_time + flickr, start_time+flickr+fl_dur]
    
    stptx = min(start_pt[0], width-100)
    stpty = min(start_pt[1], height-100)
    
    random.seed(time.time())
    # If random invisible walls are introduced.
    iwr = width 
    iwl = 0 
    iwd = height 
    iwu = 0 
    click = False #registering clicks
    tooer = False #registering early clicks
    
    # game loop
    while True:
        # event loop

        if (time.time() - start_time) >= limit:
            last_seen = ball_obj.center
            break

        # fill background colour on screen color on screen
        screen.fill(col_bg)

        # move the ball
        # Let center of the ball is (100,100) and the speed is (1,1)
        ball_obj1 = ball_obj.move(speed)
        # Now center of the ball is (101,101)
        # In this way our wall will move

        # if ball goes out of screen then change direction of movement --------------------------------------------------------------------------------------------------------
        if ball_obj.left <= 0 or ball_obj.right >= width:
            speed[0] = -speed[0]
        if ball_obj.top <= 0 or ball_obj.bottom >= height:
            speed[1] = -speed[1]
        
        # # #ball changes movement randomly -------------------------------------------------------------------------------------------------------------------------------------
        # if ball_obj1.left <= max(iwl, 0): 
        #     speed[0] = -speed[0]
        #     iwr = random.randint(min(ball_obj.right + 300, width), width)
        # if ball_obj1.right >= min(iwr, width):
        #     speed[0] = -speed[0]
        #     iwl = random.randint(0, max(ball_obj.left - 300, 0))
        # if ball_obj1.top <= max(iwu, 0):
        #     speed[1] = -speed[1]
        #     iwd = random.randint(min(ball_obj.bottom + 300, height), height)
        # if ball_obj1.bottom >= min(height,iwd):
        #     speed[1] = -speed[1]
        #     iwu = random.randint(0, max(ball_obj.top - 300, 0))

        ball_obj = ball_obj.move(speed)
        collect_pts.append(ball_obj.center)

        if (time.time() >= bound[0] and time.time() <= bound[1]) and flick_em:
            pygame.draw.circle(surface=screen, color=col_flk, center=ball_obj.center, radius=rb)  
        # draw ball at new centers that are obtained after moving ball_obj
        else:
            pygame.draw.circle(surface=screen, color=col_bb, center=ball_obj.center, radius=rb)
        
        
        timec = pygame.time.get_ticks()/1000
        clock.tick(set_fps)
        fps_disp = clock.get_fps()

        if rend_stats == True:
            font = pygame.font.SysFont('Arial', 20)
            text4 = font.render(f'Time taken: {timec}', True, (255, 255, 0))
            text5 = font.render(f'FPS: {fps_disp}', True, (255, 255, 0))
            screen.blit(text4, (0, 0))
            screen.blit(text5, (250, 0))

        # update screen
        pygame.display.flip()
        
        if time.time() <= (bound[0]):
            for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_SPACE:
                            tooer = True
        else:
            if click != True and (time.time() - start_time) <= limit:
                for event in pygame.event.get():
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_SPACE:
                            click = True
        if tooer == True:
            break
        
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    exit_disp(all_points, correct_clicks, wrong_clicks, tooerc)
                    
     
    if tooer == True:
        tooerc += 1
        if audio:
            pygame.mixer.Sound.play(wrong_ans)
    else:
        trial = trial+1
        if click == flick_em:
            correct_clicks += 1
            all_points.append([fixation,collect_pts])
            # print(rspt)
            trial_order = np.delete(trial_order, rspt, 0)
            # print(trial_order)
            if audio:
                pygame.mixer.Sound.play(right_ans)
        else:
            wrong_clicks += 1
            if audio:
                pygame.mixer.Sound.play(wrong_ans)
    

    np.random.shuffle(trial_order)            

    pygame.draw.circle(surface=screen, color=col_bg, center=ball_obj.center, radius=rb)
    pygame.display.flip()
    pygame.time.wait(1000)


    print(flick_em, click)
    print("")
                    
exit_disp(all_points, correct_clicks, wrong_clicks, tooerc)

# pygame.display.quit()
# exit()


pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
[440.0, 460.0]
[1840   80]
True True

[  80 1000]
[[array([1840,   80])
  list([(1848, 88), (1856, 96), (1864, 104), (1872, 112), (1880, 120), (1872, 128), (1864, 136), (1856, 144), (1848, 152), (1840, 160), (1832, 168), (1824, 176), (1816, 184), (1808, 192), (1800, 200), (1792, 208), (1784, 216), (1776, 224), (1768, 232), (1760, 240), (1752, 248), (1744, 256), (1736, 264), (1728, 272), (1720, 280), (1712, 288), (1704, 296), (1696, 304), (1688, 312), (1680, 320), (1672, 328), (1664, 336), (1656, 344), (1648, 352), (1640, 360), (1632, 368), (1624, 376), (1616, 384), (1608, 392), (1600, 400), (1592, 408), (1584, 416), (1576, 424), (1568, 432), (1560, 440), (1552, 448), (1544, 456), (1536, 464), (1528, 472), (1520, 480), (1512, 488), (1504, 496), (1496, 504), (1488, 512), (1480, 520), (1472, 528), (1464, 536), (1456, 544), (1448, 552), (1440, 560), (1432, 568), (1424, 576), (14

/tmp/ipykernel_7957/3244746753.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  traj = np.array(all_points)


error: video system not initialized

In [2]:
import pandas as pd
import glob
import os

path = os.getcwd() + '/Trajectories'
# print(path)
csv_files = glob.glob(os.path.join(path, "*.csv"))

for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f)
      
    # print the location and filename
    print('Location:', f)
    print('File Name:', f.split("\\")[-1])
      
    # print the content
    print('Content:')
    display(df)
    print(df)


Location: /home/yramakrishna/DeepLabCut/conda-environments/Trajectories/Trajectory_30-03-2023-10-48-40.csv
File Name: /home/yramakrishna/DeepLabCut/conda-environments/Trajectories/Trajectory_30-03-2023-10-48-40.csv
Content:


,[1840 80],"[(1848, 88), (1856, 96), (1864, 104), (1872, 112), (1880, 120), (1872, 128), (1864, 136), (1856, 144), (1848, 152), (1840, 160), (1832, 168), (1824, 176), (1816, 184), (1808, 192), (1800, 200), (1792, 208), (1784, 216), (1776, 224), (1768, 232), (1760, 240), (1752, 248), (1744, 256), (1736, 264), (1728, 272), (1720, 280), (1712, 288), (1704, 296), (1696, 304), (1688, 312), (1680, 320), (1672, 328), (1664, 336), (1656, 344), (1648, 352), (1640, 360), (1632, 368), (1624, 376), (1616, 384), (1608, 392), (1600, 400), (1592, 408), (1584, 416), (1576, 424), (1568, 432), (1560, 440), (1552, 448), (1544, 456), (1536, 464), (1528, 472), (1520, 480), (1512, 488), (1504, 496), (1496, 504), (1488, 512), (1480, 520), (1472, 528), (1464, 536), (1456, 544), (1448, 552), (1440, 560), (1432, 568), (1424, 576), (1416, 584), (1408, 592), (1400, 600), (1392, 608), (1384, 616), (1376, 624), (1368, 632), (1360, 640), (1352, 648), (1344, 656), (1336, 664), (1328, 672), (1320, 680), (1312, 688), (1304, 696), (1296, 704), (1288, 712), (1280, 720), (1272, 728), (1264, 736), (1256, 744), (1248, 752), (1240, 760), (1232, 768), (1224, 776), (1216, 784), (1208, 792), (1200, 800), (1192, 808), (1184, 816), (1176, 824), (1168, 832), (1160, 840), (1152, 848), (1144, 856), (1136, 864), (1128, 872), (1120, 880), (1112, 888), (1104, 896), (1096, 904), (1088, 912), (1080, 920), (1072, 928), (1064, 936), (1056, 944), (1048, 952), (1040, 960), (1032, 968), (1024, 976), (1016, 984), (1008, 992), (1000, 1000), (992, 1008), (984, 1016), (976, 1024), (968, 1032), (960, 1040), (952, 1032), (944, 1024), (936, 1016), (928, 1008), (920, 1000), (912, 992), (904, 984), (896, 976), (888, 968), (880, 960), (872, 952), (864, 944), (856, 936), (848, 928), (840, 920), (832, 912), (824, 904), (816, 896), (808, 888), (800, 880), (792, 872), (784, 864), (776, 856), (768, 848), (760, 840), (752, 832), (744, 824), (736, 816), (728, 808), (720, 800), (712, 792), (704, 784), (696, 776), (688, 768), (680, 760), (672, 752), (664, 744), (656, 736), (648, 728), (640, 720), (632, 712), (624, 704), (616, 696), (608, 688), (600, 680), (592, 672), (584, 664), (576, 656), (568, 648), (560, 640), (552, 632), (544, 624), (536, 616), (528, 608), (520, 600), (512, 592), (504, 584), (496, 576), (488, 568), (480, 560), (472, 552), (464, 544), (456, 536), (448, 528), (440, 520), (432, 512), (424, 504), (416, 496), (408, 488), (400, 480), (392, 472), (384, 464), (376, 456), (368, 448), (360, 440), (352, 432), (344, 424), (336, 416), (328, 408), (320, 400), (312, 392), (304, 384), (296, 376), (288, 368), (280, 360), (272, 352), (264, 344), (256, 336), (248, 328), (240, 320), (232, 312), (224, 304), (216, 296), (208, 288), (200, 280), (192, 272), (184, 264), (176, 256), (168, 248), (160, 240), (152, 232), (144, 224), (136, 216), (128, 208), (120, 200), (112, 192), (104, 184), (96, 176), (88, 168), (80, 160), (72, 152), (64, 144), (56, 136), (48, 128), (40, 120), (48, 112), (56, 104), (64, 96), (72, 88), (80, 80), (88, 72), (96, 64), (104, 56), (112, 48), (120, 40), (128, 48), (136, 56), (144, 64), (152, 72), (160, 80), (168, 88), (176, 96), (184, 104), (192, 112), (200, 120), (208, 128), (216, 136), (224, 144), (232, 152), (240, 160), (248, 168), (256, 176), (264, 184), (272, 192), (280, 200), (288, 208), (296, 216), (304, 224), (312, 232), (320, 240), (328, 248), (336, 256), (344, 264), (352, 272), (360, 280), (368, 288), (376, 296), (384, 304), (392, 312), (400, 320), (408, 328), (416, 336), (424, 344), (432, 352), (440, 360), (448, 368), (456, 376), (464, 384), (472, 392), (480, 400), (488, 408), (496, 416), (504, 424), (512, 432), (520, 440), (528, 448), (536, 456), (544, 464), (552, 472), (560, 480), (568, 488), (576, 496), (584, 504), (592, 512), (600, 520), (608, 528), (616, 536), (624, 544), (632, 552), (640, 560), (648, 568), (656, 576)]"


Empty DataFrame
Columns: [[1840   80], [(1848, 88), (1856, 96), (1864, 104), (1872, 112), (1880, 120), (1872, 128), (1864, 136), (1856, 144), (1848, 152), (1840, 160), (1832, 168), (1824, 176), (1816, 184), (1808, 192), (1800, 200), (1792, 208), (1784, 216), (1776, 224), (1768, 232), (1760, 240), (1752, 248), (1744, 256), (1736, 264), (1728, 272), (1720, 280), (1712, 288), (1704, 296), (1696, 304), (1688, 312), (1680, 320), (1672, 328), (1664, 336), (1656, 344), (1648, 352), (1640, 360), (1632, 368), (1624, 376), (1616, 384), (1608, 392), (1600, 400), (1592, 408), (1584, 416), (1576, 424), (1568, 432), (1560, 440), (1552, 448), (1544, 456), (1536, 464), (1528, 472), (1520, 480), (1512, 488), (1504, 496), (1496, 504), (1488, 512), (1480, 520), (1472, 528), (1464, 536), (1456, 544), (1448, 552), (1440, 560), (1432, 568), (1424, 576), (1416, 584), (1408, 592), (1400, 600), (1392, 608), (1384, 616), (1376, 624), (1368, 632), (1360, 640), (1352, 648), (1344, 656), (1336, 664), (1328, 672), 